In [1]:
# Imports
import pandas as pd
import numpy as np
import polars as pl
import os
import awpy
from demoparser2 import DemoParser
from awpy import Demo
from pathlib import Path
from awpy.stats import adr
from awpy.stats import kast
from awpy.stats import rating
from awpy.stats import calculate_trades
from supabase import create_client, Client
from dotenv import load_dotenv
from requests import post, get

In [2]:
folder_path = r'C:\Users\bayli\Documents\CS Demos\debugging'
file_test = r'C:\Users\bayli\Documents\CS Demos\debugging\PGL_Bucharest_2025_3dmax-vs-astralis-m1-dust2.dem'

# Creating DataFrames
df_flashes = pd.DataFrame()
df_he = pd.DataFrame()
df_infernos = pd.DataFrame()
df_smoke = pd.DataFrame()
df_kills = pd.DataFrame()
df_rounds = pd.DataFrame()
df_all_first_kills = pd.DataFrame()
df_adr = pd.DataFrame()
df_kast = pd.DataFrame()
df_util_dmg = pd.DataFrame()
df_clutches = pd.DataFrame()
df_multikills = pd.DataFrame()
team_rounds_won = pd.DataFrame()
players_id = pd.DataFrame()
df_matches = pd.DataFrame(columns=['event_id','match_name'])
i = 1
current_schema = "staging"
event_id = 3

In [3]:
def add_round_winners(ticks_df, rounds_df):
    ticks_df = ticks_df.to_pandas()
    rounds_df = rounds_df.to_pandas()

    # Makes sure the columns exists
    rounds_df['ct_team_clan_name'] = None
    rounds_df['t_team_clan_name'] = None
    rounds_df['winner_clan_name'] = None
    rounds_df['ct_team_current_equip_value'] = None
    rounds_df['t_team_current_equip_value'] = None
    rounds_df['ct_losing_streak'] = None
    rounds_df['t_losing_streak'] = None

    for idx, row in rounds_df.iterrows():
        freeze_end_tick = row['freeze_end']
        winner = row['winner']

        # Takes all corresponding entries
        first_tick_df = ticks_df[ticks_df['tick'] == freeze_end_tick]

        # Takes the name for every team
        try:
            CT_team = first_tick_df[first_tick_df['side'] == 'ct']['team_clan_name'].iloc[0]
        except IndexError:
            CT_team = None
        
        try:
            T_team = first_tick_df[first_tick_df['side'] == 't']['team_clan_name'].iloc[0]
        except IndexError:
            T_team = None

        # Takes the current equip value for every team
        try:
            CT_team_current_equip_value = first_tick_df[first_tick_df['side'] == 'ct']['current_equip_value'].sum()
        except KeyError:
            CT_team_current_equip_value = None

        try:
            T_team_current_equip_value = first_tick_df[first_tick_df['side'] == 't']['current_equip_value'].sum()
        except KeyError:
            T_team_current_equip_value = None

        # Determines the winner team name
        if winner == 'ct':
            winner_clan = CT_team
        elif winner in ['t', 'TERRORIST']:
            winner_clan = T_team
        else:
            winner_clan = None
            print(f"[!] Round {idx} - winner error: '{winner}'")
            
        # Fill Columns in the DataFrame
        rounds_df.at[idx, 'ct_team_clan_name'] = CT_team
        rounds_df.at[idx, 't_team_clan_name'] = T_team
        rounds_df.at[idx, 'winner_clan_name'] = winner_clan
        rounds_df.at[idx, 'ct_team_current_equip_value'] = CT_team_current_equip_value
        rounds_df.at[idx, 't_team_current_equip_value'] = T_team_current_equip_value


    return rounds_df

def add_losing_streaks(df: pd.DataFrame) -> pd.DataFrame:
    ct_losing_streak = []
    t_losing_streak = []

    ct_streak = 0
    t_streak = 0

    for _, row in df.iterrows():
        ct_team = row['ct_team_clan_name']
        t_team = row['t_team_clan_name']
        winner = row['winner_clan_name']
        
        if winner == ct_team:
            ct_streak = 0
            t_streak += 1
        else:  # winner == t_team
            t_streak = 0
            ct_streak += 1

        ct_losing_streak.append(ct_streak)
        t_losing_streak.append(t_streak)

    df['ct_losing_streak'] = ct_losing_streak
    df['t_losing_streak'] = t_losing_streak

    return df

def add_buy_type(row):

    if row['round_num'] in [1, 13]:
        return "Pistol", "Pistol"

    if row['ct_team_current_equip_value'] < 5000:
        ct_buy_type = "Full Eco"
    elif 5000 <= row['ct_team_current_equip_value'] < 10000:
        ct_buy_type = "Semi-Eco"
    elif 10000 <= row['ct_team_current_equip_value'] < 20000:
        ct_buy_type = "Semi-Buy"
    elif row['ct_team_current_equip_value'] >= 20000:
        ct_buy_type = "Full Buy"
    else:
        ct_buy_type = "Unknown"

    if row['t_team_current_equip_value'] < 5000:
        t_buy_type = "Full Eco"
    elif 5000 <= row['t_team_current_equip_value'] < 10000:
        t_buy_type = "Semi-Eco"
    elif 10000 <= row['t_team_current_equip_value'] < 20000:
        t_buy_type = "Semi-Buy"
    elif row['t_team_current_equip_value'] >= 20000:
        t_buy_type = "Full Buy"
    else:
        t_buy_type = "Unknown"

    return ct_buy_type, t_buy_type

def calculate_advantage_5v4(rounds_df, first_kills_df):

    # Makes sure the columns exists
    rounds_df['advantage_5v4'] = None

    # Checks what team got the first kill
    for idx, row in rounds_df.iterrows():
        round_num = row['round_num']

        # Filters the first kills DataFrame for the current round
        first_kill = first_kills_df[first_kills_df['round_num'] == round_num]

        if not first_kill.empty:
            # Gets the team that made the first kill
            killer_team = first_kill.iloc[0]['attacker_side']

            # Defines the advantage based on the killer team
            if killer_team == 'ct':
                rounds_df.at[idx, 'advantage_5v4'] = 'ct'
            elif killer_team == 't':
                rounds_df.at[idx, 'advantage_5v4'] = 't'

    return rounds_df

def insert_table(df, current_schema, table_name, conflict_cols):
    for row in df.to_dict(orient="records"):
        supabase.schema(current_schema).table(table_name).upsert(row, on_conflict=conflict_cols).execute()

def insert_or_update_player_history(players_df):
    for _, row in players_df.iterrows():
        steam_id = row["steam_id"]
        team_id = row["team_id"]
        
        # Finds out if the players have changed teams
        player_history_data = {
            "steam_id": steam_id,
            "team_id": team_id
        }
        # Updates the player history table with the new data
        supabase.table("player_history").upsert(
            player_history_data, 
            on_conflict=["steam_id", "team_id"]
        ).execute()

def rounds_correction(df: pl.DataFrame) -> pl.DataFrame:
    freeze_end_is_null = df.select(pl.col("freeze_end").first().is_null()).item()

    if freeze_end_is_null:
        df = df.with_columns(
            (pl.col("round_num") - 1).alias("round_num")
        )
        
    return df.filter(pl.col("freeze_end").is_not_null())

def fetch_all_rows(current_schema, table_name, page_size=1000):
    offset = 0
    all_data = []

    while True:
        response = supabase.schema(current_schema).table(table_name).select("*").range(offset, offset + page_size - 1).execute()
        data = response.data
        if not data:
            break
        all_data.extend(data)
        offset += page_size

    return all_data

def calculate_clutches(dem) -> pl.DataFrame:
    clutches_data = []

    # Apply rounds_correction to dem.rounds
    corrected_rounds = rounds_correction(dem.rounds)

    all_players = dem.ticks.select(["steamid", "name"]).unique()
    player_name_map = {row["steamid"]: row["name"] for row in all_players.iter_rows(named=True)}

    for round_info in corrected_rounds.iter_rows(named=True):
        round_num = round_info['round_num']
        round_start_tick = round_info['start']
        round_end_tick = round_info['end']
        winning_team = round_info['winner']

        round_ticks_df = dem.ticks.filter(
            (pl.col("tick") >= round_start_tick) & (pl.col("tick") <= round_end_tick)
        )
        round_kills_df = dem.kills.filter(
            (pl.col("round_num") == round_num) &
            (pl.col("tick") >= round_start_tick) & (pl.col("tick") <= round_end_tick)
        ).sort("tick")

        alive_counts_per_tick = round_ticks_df.group_by("tick").agg(
            (pl.when(pl.col("side") == "ct").then(pl.col("is_alive")).otherwise(0)).sum().alias("ct_alive"),
            (pl.when(pl.col("side") == "t").then(pl.col("is_alive")).otherwise(0)).sum().alias("t_alive")
        )
        
        players_alive_at_tick = round_ticks_df.group_by("tick").agg(
            pl.struct(["steamid", "is_alive", "side"]).alias("players_state")
        )
        
        merged_round_data = alive_counts_per_tick.join(
            players_alive_at_tick, on="tick", how="left"
        ).sort("tick")


        clutch_active = False
        clutcher_steamid = None
        clutcher_team_side = None
        clutch_start_tick = None
        opponents_at_start = 0

        for i, tick_data in enumerate(merged_round_data.iter_rows(named=True)):
            current_tick = tick_data['tick']
            ct_alive = tick_data['ct_alive']
            t_alive = tick_data['t_alive']
            players_state = tick_data['players_state']
            
            is_clutch_condition_met = (ct_alive == 1 and t_alive >= 2) or \
                                        (t_alive == 1 and ct_alive >= 2)

            clutcher_is_alive_this_tick = False
            if clutch_active and clutcher_steamid:
                for p_state in players_state:
                    if p_state['steamid'] == clutcher_steamid and p_state['is_alive']:
                        clutcher_is_alive_this_tick = True
                        break

            if is_clutch_condition_met:
                if not clutch_active: # Start of a NEW clutch
                    clutch_active = True
                    clutch_start_tick = current_tick
                    
                    if ct_alive == 1:
                        clutcher_team_side = 'ct'
                        opponents_at_start = t_alive # Capture opponents at the start
                    else: # t_alive == 1
                        clutcher_team_side = 't'
                        opponents_at_start = ct_alive # Capture opponents at the start
                    
                    for p_state in players_state:
                        if p_state['is_alive'] and p_state['side'] == clutcher_team_side:
                            clutcher_steamid = p_state['steamid']
                            break
            
            else: # Clutch condition (1vX, X>=2) is NO longer met in this tick
                if clutch_active: # But there was an active clutch in the previous tick
                    # The clutch ends if the clutcher dies
                    if not clutcher_is_alive_this_tick:
                        clutch_end_tick = current_tick # Clutch ends with the clutcher's death
                        
                        # --- CLUTCH OUTCOME: "lost" ---
                        clutch_outcome = "lost" # Clutcher died

                        final_clutch_kills = 0
                        if clutcher_steamid:
                            final_clutch_kills = round_kills_df.filter(
                                (pl.col("tick") >= clutch_start_tick) &
                                (pl.col("tick") <= clutch_end_tick) &
                                (pl.col("attacker_steamid") == clutcher_steamid)
                            ).shape[0]

                        # Determine if the clutcher's team won the round
                        round_team_won = (clutcher_team_side == winning_team.lower()) 

                        # Clutcher did not survive the round if they died to end the clutch
                        clutcher_survived_round = False

                        clutches_data.append({
                            "round_num": round_num,
                            "clutcher_steamid": clutcher_steamid,
                            "clutcher_name": player_name_map.get(clutcher_steamid, "Unknown"),
                            "clutcher_team_side": clutcher_team_side,
                            "opponents_at_start": opponents_at_start,
                            "clutch_start_tick": clutch_start_tick,
                            "clutch_end_tick": clutch_end_tick,
                            "clutch_kills": final_clutch_kills,
                            "clutch_outcome": clutch_outcome,
                            "round_won": round_team_won,
                            "clutcher_survived_round": clutcher_survived_round
                        })
                        
                        clutch_active = False
                        clutcher_steamid = None
                        clutcher_team_side = None
                        clutch_start_tick = None
                        opponents_at_start = 0

        # --- Round end processing logic ---
        if clutch_active:
            clutch_end_tick = round_end_tick 
            
            final_clutch_kills = 0
            if clutcher_steamid:
                final_clutch_kills = round_kills_df.filter(
                    (pl.col("tick") >= clutch_start_tick) &
                    (pl.col("tick") <= clutch_end_tick) &
                    (pl.col("attacker_steamid") == clutcher_steamid)
                ).shape[0]

            # Determine if the clutcher's team won the round
            round_team_won = (clutcher_team_side == winning_team.lower())

            # NOVO: Determinar o clutch_outcome para o final da rodada
            if round_team_won:
                # --- CLUTCH OUTCOME: "won" ---
                clutch_outcome = "won" # Clutcher's team won the round
            else:
                # --- CLUTCH OUTCOME: "save" ---
                clutch_outcome = "save" # Clutcher survived, but team lost the round (e.g., time expired, bomb detonated for opponents)
            
            # Determine if the clutcher survived the round (always True if we reach this block)
            # We already know clutcher is alive at this point because `clutch_active` is True
            # and they weren't caught by the `if not clutcher_is_alive_this_tick` block.
            clutcher_survived_round = True # If clutch_active is true at end of round, clutcher survived.

            clutches_data.append({
                "round_num": round_num,
                "clutcher_steamid": clutcher_steamid,
                "clutcher_name": player_name_map.get(clutcher_steamid, "Unknown"),
                "clutcher_team_side": clutcher_team_side,
                "opponents_at_start": opponents_at_start,
                "clutch_start_tick": clutch_start_tick,
                "clutch_end_tick": clutch_end_tick,
                "clutch_kills": final_clutch_kills,
                "clutch_outcome": clutch_outcome, # Renomeado
                "round_won": round_team_won,
                "clutcher_survived_round": clutcher_survived_round
            })
            
            clutch_active = False
            clutcher_steamid = None
            clutcher_team_side = None
            clutch_start_tick = None
            opponents_at_start = 0

    clutches_df = pl.DataFrame(clutches_data)
    return clutches_df

def calculate_multikill_rounds(dem) -> pl.DataFrame:
    # Ensure 'dem.kills' is not empty before proceeding
    if dem.kills is None or dem.kills.shape[0] == 0:
        return pl.DataFrame({"steamid": pl.Series(dtype=pl.UInt64),
                             "2k": pl.Series(dtype=pl.UInt32),
                             "3k": pl.Series(dtype=pl.UInt32),
                             "4k": pl.Series(dtype=pl.UInt32),
                             "5k": pl.Series(dtype=pl.UInt32)})

    kills_per_player_per_round = dem.kills.group_by(["round_num", "attacker_steamid"]).agg(
        pl.len().alias("kills_in_round")
    )

    multikill_counts = kills_per_player_per_round.group_by("attacker_steamid").agg(
        pl.when(pl.col("kills_in_round") == 2).then(1).otherwise(0).sum().alias("2k"),
        pl.when(pl.col("kills_in_round") == 3).then(1).otherwise(0).sum().alias("3k"),
        pl.when(pl.col("kills_in_round") == 4).then(1).otherwise(0).sum().alias("4k"),
        pl.when(pl.col("kills_in_round") == 5).then(1).otherwise(0).sum().alias("5k")
    )

    multikill_counts = multikill_counts.rename({"attacker_steamid": "steamid"})

    return multikill_counts

In [4]:
# Full Loop
for file_name in os.listdir(folder_path):
    if file_name.endswith('.dem'):

        file_path = os.path.join(folder_path, file_name)
        dem = Demo(file_path)
        dem.parse(player_props=["team_clan_name","total_rounds_played", "current_equip_value", "ct_losing_streak", "t_losing_streak", "is_alive"])

        # Gets all the Players' steam_ids
        this_file_players_id = dem.events.get('player_spawn')
        this_file_players_id = this_file_players_id.with_columns(
            this_file_players_id['user_steamid'].cast(pl.Utf8)
        )
        this_file_players_id = this_file_players_id.to_pandas()
        this_file_players_id = this_file_players_id[['user_steamid', 'user_name']].drop_duplicates()
        players_id = pd.concat([players_id, this_file_players_id], ignore_index=True)
        players_id = players_id[['user_steamid', 'user_name']].drop_duplicates()

        # Grenades Data
        # Makes that the data frame is not empty and that the columns are in the right format
        this_file_flashes = dem.events.get('flashbang_detonate', pl.DataFrame())
        if this_file_flashes is not None and len(this_file_flashes) > 0:
            this_file_flashes = this_file_flashes.with_columns(
                this_file_flashes['user_steamid'].cast(pl.Utf8)
            )
        this_file_he = dem.events.get('hegrenade_detonate', pl.DataFrame())
        if this_file_he is not None and len(this_file_he) > 0:
            this_file_he = this_file_he.with_columns(
                this_file_he['user_steamid'].cast(pl.Utf8)
            )
        this_file_infernos = dem.events.get('inferno_startburn', pl.DataFrame())
        if this_file_infernos is not None and len(this_file_infernos) > 0:
            this_file_infernos = this_file_infernos.with_columns(
                this_file_infernos['user_steamid'].cast(pl.Utf8)
            )
        this_file_smoke = dem.events.get('smokegrenade_detonate', pl.DataFrame())
        if this_file_smoke is not None and len(this_file_smoke) > 0:
            this_file_smoke = this_file_smoke.with_columns(
                this_file_smoke['user_steamid'].cast(pl.Utf8)
            )
        this_file_util_dmg = dem.events.get('player_hurt', pl.DataFrame())
        if this_file_util_dmg is not None and len(this_file_util_dmg) > 0:
            this_file_util_dmg = this_file_util_dmg.with_columns(
                this_file_util_dmg['attacker_steamid'].cast(pl.Utf8)
            )
        util_dmg = this_file_util_dmg.filter(
            (this_file_util_dmg["weapon"] == "hegrenade") |
            (this_file_util_dmg["weapon"] == "molotov")   |
            (this_file_util_dmg["weapon"] == "inferno")
        )
        # Makes sure that the data frames are not empty, converts them to pandas and appends them to the main data frame
        if this_file_flashes is not None and len(this_file_flashes) > 0:
            df_flashes = pd.concat([df_flashes, this_file_flashes.to_pandas()], ignore_index=True)
        if this_file_he is not None and len(this_file_he) > 0:
            df_he = pd.concat([df_he, this_file_he.to_pandas()], ignore_index=True)
        if this_file_infernos is not None and len(this_file_infernos) > 0:
            df_infernos = pd.concat([df_infernos, this_file_infernos.to_pandas()], ignore_index=True)
        if this_file_smoke is not None and len(this_file_smoke) > 0:
            df_smoke = pd.concat([df_smoke, this_file_smoke.to_pandas()], ignore_index=True)
        if this_file_util_dmg is not None and len(this_file_util_dmg) > 0:
            df_util_dmg = pd.concat([df_util_dmg, this_file_util_dmg.to_pandas()], ignore_index=True)

        # Opening Kills Data
        this_file_df_kills = awpy.stats.calculate_trades(demo=dem)
        this_file_df_kills = this_file_df_kills.with_columns(
            this_file_df_kills['attacker_steamid'].cast(pl.Utf8),
            this_file_df_kills['assister_steamid'].cast(pl.Utf8),
            this_file_df_kills['victim_steamid'].cast(pl.Utf8)
        )
        this_file_df_kills = this_file_df_kills.to_pandas()
        first_kills = this_file_df_kills.sort_values(by=['round_num', 'tick'])
        first_kills = first_kills.groupby('round_num').first().reset_index()
        df_all_first_kills = pd.concat([df_all_first_kills, first_kills], ignore_index=True)

        # Creates Match Table
        folder_name = os.path.basename(os.path.dirname(file_path))
        file_name = file_name.replace(f"{folder_name}_", "")
        df_matches = pd.concat([df_matches, pd.DataFrame({'match_name': [file_name], 'event_id': [event_id]})], ignore_index=True)

        # Rounds Data
        this_file_df_ticks = dem.ticks
        this_file_df_rounds = dem.rounds
        this_file_df_rounds = rounds_correction(this_file_df_rounds)
        this_file_df_rounds = add_round_winners(this_file_df_ticks,this_file_df_rounds)
        this_file_df_rounds = add_losing_streaks(this_file_df_rounds)
        this_file_df_rounds[['ct_buy_type', 't_buy_type']] = this_file_df_rounds.apply(add_buy_type, axis=1, result_type='expand')
        first_kills = this_file_df_kills.sort_values(by=['round_num', 'tick'])
        first_kills = first_kills.groupby('round_num').first().reset_index()
        df_all_first_kills = pd.concat([df_all_first_kills, first_kills], ignore_index=True)   

        this_file_df_rounds = calculate_advantage_5v4(this_file_df_rounds, df_all_first_kills)
        df_rounds = pd.concat([df_rounds, this_file_df_rounds], ignore_index=True)
        df_rounds['event_id'] = event_id
        # Creates rounds won columns
        this_file_team_rounds_won = this_file_df_rounds.groupby('winner_clan_name').agg(
            total_rounds_won=('winner_clan_name', 'size'),
            t_rounds_won=('winner', lambda x: (x == 'ct').sum()),
            ct_rounds_won=('winner', lambda x: (x == 't').sum())
        ).reset_index()
        this_file_team_rounds_won.columns = ['team_clan_name', 'total_rounds_won','t_rounds_won', 'ct_rounds_won']
        team_rounds_won = pd.concat([team_rounds_won,this_file_team_rounds_won], ignore_index=True)
        df_kills = pd.concat([df_kills,this_file_df_kills], ignore_index=True)

        # ADR Data
        this_file_adr = awpy.stats.adr(demo=dem)
        this_file_adr = this_file_adr.with_columns(
            this_file_adr['steamid'].cast(pl.Utf8)
        )
        this_file_adr = this_file_adr.to_pandas()
        this_file_adr = this_file_adr.drop(['adr', 'name'], axis=1)
        df_adr = pd.concat([df_adr, this_file_adr], ignore_index=True)
        df_adr = df_adr.groupby(['steamid','side'], as_index=False).sum()
        df_adr = df_adr[df_adr['side'] != 'all']

        # KAST Data
        this_file_kast = awpy.stats.kast(demo=dem)
        this_file_kast = this_file_kast.with_columns(
            this_file_kast['steamid'].cast(pl.Utf8)
        )
        this_file_kast = this_file_kast.to_pandas()
        this_file_kast = this_file_kast.drop(['kast', 'name'], axis=1)
        df_kast = pd.concat([df_kast, this_file_kast], ignore_index=True)
        df_kast = df_kast.groupby(['steamid','side'], as_index=False).sum()
        df_kast = df_kast[df_kast['side'] != 'all']

        # Creates Clutches Dataframe
        this_file_clutches = calculate_clutches(dem)
        this_file_clutches = this_file_clutches.with_columns(
            this_file_clutches['clutcher_steamid'].cast(pl.Utf8)
        )
        this_file_clutches = this_file_clutches.to_pandas()
        df_clutches = pd.concat([df_clutches,this_file_clutches], ignore_index=True)
        df_clutches = df_clutches.drop(['clutcher_name', 'clutch_start_tick', 'clutch_end_tick'], axis=1)
        df_clutches['file_id'] = event_id

        # Creates Multikills Dataframe
        this_file_multikills = calculate_multikill_rounds(dem)
        this_file_multikills = this_file_multikills.with_columns(
            this_file_multikills['steamid'].cast(pl.Utf8)
        )
        this_file_multikills = this_file_multikills.to_pandas()
        df_multikills = pd.concat([df_multikills, this_file_multikills], ignore_index=True)

        print(f"{i}: Processed {file_name}")
        i = i + 1


1: Processed PGL_Bucharest_2025_3dmax-vs-astralis-m1-dust2.dem


In [5]:
player_kills = df_kills.groupby('attacker_steamid').agg(
    kills=('attacker_steamid', 'size'),
    headshots=('headshot', lambda x: (x == 1).sum()),
    wallbang_kills=('penetrated', lambda x: (x == 1).sum()),
    assisted_flashes=('assistedflash', lambda x: (x == 1).sum()),
    trade_kills=('was_traded', lambda x: (x == 1).sum()),
    no_scope=('noscope', lambda x: (x == 1).sum()),
    through_smoke=('thrusmoke', lambda x: (x == 1).sum()),
    airborne_kills=('attackerinair', lambda x: (x == 1).sum()),
    blind_kills=('attackerblind', lambda x: (x == 1).sum()),
    victim_blind_kills=('assistedflash', lambda x: (x == 1).sum()),
    attacker_team_clan_name=('attacker_team_clan_name', 'first'),
    awp_kills=('weapon', lambda x: (x == 'awp').sum()),
    pistol_kills=('weapon', lambda x: x.isin(['glock', 'usp_silencer', 'p250', 'p2000' , 'tec9', 'cz75_auto', 'fiveseven', 'deagle', 'elite', 'revolver']).sum())
).reset_index()
player_kills.rename(columns={'attacker_steamid': 'steam_id'}, inplace=True)
player_kills.rename(columns={'attacker_team_clan_name': 'team_clan_name'}, inplace=True)

player_assists = df_kills.groupby('assister_steamid').agg(
    assists=('assister_steamid', 'size')
).reset_index()
player_assists.rename(columns={'assister_steamid': 'steam_id'}, inplace=True)

player_deaths = df_kills.groupby('victim_steamid').agg(
    deaths=('victim_steamid', 'size'),
    trade_deaths=('was_traded', lambda x: (x == 1).sum())
).reset_index()
player_deaths.rename(columns={'victim_steamid': 'steam_id'}, inplace=True)

players = player_kills.merge(player_assists, on='steam_id', how='left').merge(player_deaths, on='steam_id', how='left')
players['steam_id'] = player_kills['steam_id'].astype('int64')
players['kd'] = players['kills'] / players['deaths']
players['k_d_diff'] = players['kills'] - players['deaths']


# ADR Total
adr_total = df_adr.groupby('steamid').agg({'dmg': 'sum', 'n_rounds': 'sum'})
adr_total['adr_total'] = adr_total['dmg'] / adr_total['n_rounds']
# ADR CT
df_ct = df_adr[df_adr['side'] == 'ct']
adr_ct = df_ct.groupby('steamid').agg({'dmg': 'sum', 'n_rounds': 'sum'})
adr_ct['adr_ct_side'] = adr_ct['dmg'] / adr_ct['n_rounds']
# ADR T
df_t = df_adr[df_adr['side'] == 't']
adr_t = df_t.groupby('steamid').agg({'dmg': 'sum', 'n_rounds': 'sum'})
adr_t['adr_t_side'] = adr_t['dmg'] / adr_t['n_rounds']
# ADR Data Frame
df_adr = pd.DataFrame({
    'steamid': adr_total.index,
    'adr_total': adr_total['adr_total'],
    'adr_ct_side': adr_ct['adr_ct_side'].reindex(adr_total.index, fill_value=0),
    'adr_t_side': adr_t['adr_t_side'].reindex(adr_total.index, fill_value=0),
}).reset_index(drop=True)
# Merge ADR
df_adr.rename(columns={'steamid': 'steam_id'}, inplace=True)
df_adr['steam_id'] = df_adr['steam_id'].astype('int64')
players = players.merge(df_adr, on='steam_id', how='left')

# KAST Total
kast_total = df_kast.groupby('steamid').agg({'kast_rounds': 'sum', 'n_rounds': 'sum'})
kast_total['kast_total'] = kast_total['kast_rounds'] / kast_total['n_rounds']
# KAST CT
df_ct = df_kast[df_kast['side'] == 'ct']
kast_ct = df_ct.groupby('steamid').agg({'kast_rounds': 'sum', 'n_rounds': 'sum'})
kast_ct['kast_ct_side'] = kast_ct['kast_rounds'] / kast_ct['n_rounds']
# KAST T
df_t = df_kast[df_kast['side'] == 't']
kast_t = df_t.groupby('steamid').agg({'kast_rounds': 'sum', 'n_rounds': 'sum'})
kast_t['kast_t_side'] = kast_t['kast_rounds'] / kast_t['n_rounds']
# KAST Data Frame
df_kast = pd.DataFrame({
    'steamid': kast_total.index,
    'kast_total': kast_total['kast_total'],
    'kast_ct_side': kast_ct['kast_ct_side'].reindex(kast_total.index, fill_value=0),
    'kast_t_side': kast_t['kast_t_side'].reindex(kast_total.index, fill_value=0),
}).reset_index(drop=True)
# Merge KAST
df_kast.rename(columns={'steamid': 'steam_id'}, inplace=True)
df_kast['steam_id'] = df_kast['steam_id'].astype('int64')
players = players.merge(df_kast, on='steam_id', how='left')

# Rounds won Data
team_rounds_won = team_rounds_won.groupby('team_clan_name').agg(
    total_rounds_won=('total_rounds_won', 'sum'),
    t_rounds_won=('t_rounds_won', 'sum'),
    ct_rounds_won=('ct_rounds_won', 'sum')
).reset_index()

players = players.merge(team_rounds_won, on='team_clan_name', how='left')

# Opening Duels
# Creates the player table with all the kills raw data grouped for every match in a single table
opening_kills = df_all_first_kills.groupby('attacker_steamid').agg(
    first_kills=('attacker_steamid', 'size'),
    ct_first_kills=('attacker_side', lambda x: (x == 'ct').sum()),
    t_first_kills=('attacker_side', lambda x: (x == 't').sum())
).reset_index()
opening_kills.rename(columns={'attacker_steamid': 'steam_id'}, inplace=True)
opening_kills['steam_id'] = opening_kills['steam_id'].astype('int64')

opening_deaths = df_all_first_kills.groupby('victim_steamid').agg(
    first_deaths=('victim_steamid', 'size'),  
    ct_first_deaths=('victim_side', lambda x: (x == 'ct').sum()),  
    t_first_deaths=('victim_side', lambda x: (x == 't').sum())  
).reset_index()
opening_deaths.rename(columns={'victim_steamid': 'steam_id'}, inplace=True)
opening_deaths['steam_id'] = opening_deaths['steam_id'].astype('int64')

player_first_kills = opening_kills.merge(opening_deaths, on='steam_id', how='left')
players = players.merge(player_first_kills, on='steam_id', how='left')

# Group all the grenades data in separated dataframes
df_flashes.rename(columns={'user_total_rounds_played': 'round'}, inplace=True)
df_flashes['round'] = df_flashes['round'] + 1
df_all_flashes = df_flashes.groupby('user_steamid').agg(
    flashes_thrown=('user_steamid', 'size'),
    ct_flashes_thrown=('user_side', lambda x: (x == 'ct').sum()),
    t_flashes_thrown=('user_side', lambda x: (x == 't').sum()),
    flashes_thrown_in_pistol_round=('round', lambda x: ((x == 1) | (x == 13)).sum()),
).reset_index()
df_all_flashes.rename(columns={'user_steamid': 'steam_id'}, inplace=True)
df_all_flashes['steam_id'] = df_all_flashes['steam_id'].astype('int64')

df_he.rename(columns={'user_total_rounds_played': 'round'}, inplace=True)
df_he['round'] = df_he['round'] + 1
df_all_he = df_he.groupby('user_steamid').agg(
    he_thrown=('user_steamid', 'size'),
    ct_he_thrown=('user_side', lambda x: (x == 'ct').sum()),
    t_he_thrown=('user_side', lambda x: (x == 't').sum()),
    he_thrown_in_pistol_round=('round', lambda x: ((x == 1) | (x == 13)).sum())
).reset_index()
df_all_he.rename(columns={'user_steamid': 'steam_id'}, inplace=True)
df_all_he['steam_id'] = df_all_he['steam_id'].astype('int64')

df_infernos.rename(columns={'user_total_rounds_played': 'round'}, inplace=True)
df_infernos['round'] = df_infernos['round'] + 1
df_all_infernos = df_infernos.groupby('user_steamid').agg(
    infernos_thrown=('user_steamid', 'size'),
    ct_infernos_thrown=('user_side', lambda x: (x == 'ct').sum()),
    t_infernos_thrown=('user_side', lambda x: (x == 't').sum()),
    infernos_thrown_in_pistol_round=('round', lambda x: ((x == 1) | (x == 13)).sum())
).reset_index()
df_all_infernos.rename(columns={'user_steamid': 'steam_id'}, inplace=True)
df_all_infernos['steam_id'] = df_all_infernos['steam_id'].astype('int64')

df_smoke.rename(columns={'user_total_rounds_played': 'round'}, inplace=True)
df_smoke['round'] = df_smoke['round'] + 1
df_all_smokes = df_smoke.groupby('user_steamid').agg(
    smokes_thrown=('user_steamid', 'size'),
    ct_smokes_thrown=('user_side', lambda x: (x == 'ct').sum()),
    t_smokes_thrown=('user_side', lambda x: (x == 't').sum()),
    smokes_thrown_in_pistol_round=('round', lambda x: ((x == 1) | (x == 13)).sum())
).reset_index()
df_all_smokes.rename(columns={'user_steamid': 'steam_id'}, inplace=True)
df_all_smokes['steam_id'] = df_all_smokes['steam_id'].astype('int64')

# Filter the utility damage data to only include the relevant columns and group by player
df_util_dmg = df_util_dmg[(df_util_dmg["weapon"] == "hegrenade") | (df_util_dmg["weapon"] == "molotov") | (df_util_dmg["weapon"] == "inferno")]
df_util_dmg.loc[:, "total_dmg"] = df_util_dmg["dmg_health"]
df_util_dmg = df_util_dmg.groupby("attacker_steamid").agg(
    total_util_dmg=("total_dmg", "sum"),
    ct_total_util_dmg=("total_dmg", lambda x: x[df_util_dmg.loc[x.index, "attacker_side"] == "ct"].sum()),
    t_total_util_dmg=("total_dmg", lambda x: x[df_util_dmg.loc[x.index, "attacker_side"] == "t"].sum())
).reset_index()
df_util_dmg.rename(columns={'attacker_steamid': 'steam_id'}, inplace=True)
df_util_dmg['steam_id'] = df_util_dmg['steam_id'].astype('int64')

# # Group the multikills data
# df_multikills = df_multikills.groupby('steam_id')[['2k', '3k', '4k', '5k']].sum().reset_index()
# df_multikills['steam_id'] = df_multikills['steam_id'].astype('int64')

# Merges all the current data in a single dataframe
players = players.merge(df_all_flashes, on='steam_id', how='left')
players = players.merge(df_all_he, on='steam_id', how='left')
players = players.merge(df_all_infernos, on='steam_id', how='left')
players = players.merge(df_all_smokes, on='steam_id', how='left')
# players = players.merge(df_multikills, on='steam_id', how='left')
players['util_in_pistol_round'] = players['flashes_thrown_in_pistol_round'] + players['he_thrown_in_pistol_round'] + players['infernos_thrown_in_pistol_round'] +  players['smokes_thrown_in_pistol_round']
players['total_util_thrown'] = players['flashes_thrown'] + players['he_thrown'] + players['infernos_thrown'] +  players['smokes_thrown']
players = players.merge(df_util_dmg, on='steam_id', how='left')

players_id.rename(columns={'user_steamid': 'steam_id'}, inplace=True)
players_id['steam_id'] = players_id['steam_id'].fillna(0).astype('int64')
players_id['steam_id'] = players_id['steam_id'].astype('int64')
players = players.merge(players_id, on='steam_id', how='left')
players = players[["steam_id", "user_name"] + [col for col in players.columns if col not in ["steam_id", "user_name"]]]

# Creates the teams table
teams = pd.DataFrame({'team_clan_name': pd.concat([df_rounds['ct_team_clan_name'], df_rounds['t_team_clan_name']]).dropna().unique()})


In [7]:
df_rounds

,round_num,start,end,official_end,freeze_end,winner,reason,bomb_plant,bomb_site,ct_team_clan_name,t_team_clan_name,winner_clan_name,ct_team_current_equip_value,t_team_current_equip_value,ct_losing_streak,t_losing_streak,ct_buy_type,t_buy_type,advantage_5v4,event_id
0,1,1024,5860,6308,2304,ct,t_killed,NaN,not_planted,3DMAX,Astralis,3DMAX,4500.0,4500.0,0,1,Pistol,Pistol,ct,3
1,2,6308,12465,12913,7588,ct,t_killed,NaN,not_planted,3DMAX,Astralis,3DMAX,19650.0,10100.0,0,2,Semi-Buy,Semi-Buy,t,3
2,3,12913,20935,21383,14193,ct,t_killed,NaN,not_planted,3DMAX,Astralis,3DMAX,22950.0,4200.0,0,3,Full Buy,Full Eco,ct,3
3,4,21383,25154,25602,22663,t,ct_killed,NaN,not_planted,3DMAX,Astralis,Astralis,26100.0,23750.0,1,0,Full Buy,Full Buy,ct,3
4,5,25602,34242,34690,26882,ct,time_ran_out,NaN,not_planted,3DMAX,Astralis,3DMAX,27200.0,25600.0,0,1,Full Buy,Full Buy,t,3
5,6,34690,45621,46069,38763,ct,bomb_defused,44404.0,bombsite_b,3DMAX,Astralis,3DMAX,27150.0,21500.0,0,2,Full Buy,Full Buy,t,3
6,7,46069,58116,58564,49939,ct,bomb_defused,55721.0,bombsite_b,3DMAX,Astralis,3DMAX,26300.0,9300.0,0,3,Full Buy,Semi-Eco,t,3
7,8,58564,67350,67798,59844,t,ct_killed,65061.0,bombsite_b,3DMAX,Astralis,Astralis,26750.0,27350.0,1,0,Full Buy,Full Buy,t,3
8,9,67798,76438,76886,69078,ct,time_ran_out,NaN,not_planted,3DMAX,Astralis,3DMAX,7250.0,27450.0,0,1,Semi-Eco,Full Buy,t,3
9,10,76886,87398,87846,80837,t,ct_killed,86409.0,bombsite_b,3DMAX,Astralis,Astralis,28050.0,25950.0,1,0,Full Buy,Full Buy,ct,3


In [12]:
df_multikills = df_multikills.groupby('steamid')[['2k', '3k', '4k', '5k']].sum()
df_multikills

,2k,3k,4k,5k
steamid,,,,
76561197989430253,3.0,3.0,1.0,0.0
76561197991272318,8.0,4.0,0.0,0.0
76561197997351207,13.0,2.0,0.0,0.0
76561198013243326,7.0,2.0,0.0,0.0
76561198050250233,4.0,2.0,0.0,0.0
76561198068422762,6.0,4.0,1.0,0.0
76561198176878303,7.0,0.0,1.0,0.0
76561198201620490,8.0,0.0,0.0,0.0
76561198246607476,8.0,2.0,1.0,1.0


In [5]:
dem = Demo(file_test)
dem.parse(player_props=["team_clan_name","total_rounds_played", "current_equip_value", "round_num", "is_alive"])

In [ ]:
pl.Config.set_tbl_rows(10)

In [ ]:
dem.rounds

In [6]:
dem.kills

assistedflash,assister_X,assister_Y,assister_Z,assister_current_equip_value,assister_health,assister_is_alive,assister_place,assister_name,assister_steamid,assister_team_clan_name,assister_side,assister_total_rounds_played,attacker_X,attacker_Y,attacker_Z,attacker_current_equip_value,attacker_health,attacker_is_alive,attacker_place,attacker_name,attacker_steamid,attacker_team_clan_name,attacker_side,attacker_total_rounds_played,attackerblind,attackerinair,ct_team_clan_name,ct_side,distance,dmg_armor,dmg_health,dominated,headshot,hitgroup,noreplay,noscope,penetrated,revenge,t_team_clan_name,t_side,thrusmoke,tick,total_rounds_played,victim_X,victim_Y,victim_Z,victim_current_equip_value,victim_health,victim_is_alive,victim_place,victim_name,victim_steamid,victim_team_clan_name,victim_side,victim_total_rounds_played,weapon,weapon_fauxitemid,weapon_itemid,weapon_originalowner_xuid,wipe,round_num
bool,f32,f32,f32,f64,f64,bool,str,str,u64,str,str,f64,f32,f32,f32,f64,f64,bool,str,str,u64,str,str,f64,bool,bool,str,str,f32,i32,i32,i32,bool,str,bool,bool,i32,i32,str,str,bool,i32,i32,f32,f32,f32,u32,i32,bool,str,str,u64,str,str,i32,str,str,str,str,i32,u32
false,null,null,null,null,null,null,null,null,null,null,null,null,2164.376709,-74.899841,120.676041,850.0,100.0,true,"""BombsiteA""","""rain""",76561197997351207,"""FaZe Clan""","""t""",0.0,false,false,"""Natus Vincere""","""ct""",26.066914,5,10,0,false,"""chest""",false,false,0,0,"""FaZe Clan""","""t""",false,9199,0,2441.68457,911.722412,168.82196,850,5,true,"""BombsiteA""","""b1t""",76561198246607476,"""Natus Vincere""","""ct""",0,"""glock""","""17293822569105195012""","""39885972092""","""""",0,1
false,null,null,null,null,null,null,null,null,null,null,null,null,2520.411377,-218.367371,94.028725,850.0,100.0,true,"""Pit""","""iM""",76561198050250233,"""Natus Vincere""","""ct""",0.0,false,false,"""Natus Vincere""","""ct""",11.157001,0,129,0,true,"""head""",false,false,0,0,"""FaZe Clan""","""t""",false,9326,0,2125.074951,-325.528381,247.309143,850,100,true,"""Balcony""","""ropz""",76561197991272318,"""FaZe Clan""","""t""",0,"""usp_silencer""","""17293822569135734845""","""39856854336""","""""",0,1
false,null,null,null,null,null,null,null,null,null,null,null,null,2033.328003,212.213135,209.03125,850.0,100.0,true,"""BombsiteA""","""rain""",76561197997351207,"""FaZe Clan""","""t""",0.0,false,false,"""Natus Vincere""","""ct""",14.471786,6,11,0,false,"""neck""",false,false,0,0,"""FaZe Clan""","""t""",false,9414,0,2495.184082,-99.897949,99.591751,850,1,true,"""Pit""","""iM""",76561198050250233,"""Natus Vincere""","""ct""",0,"""glock""","""17293822569105195012""","""39885972092""","""""",0,1
false,null,null,null,null,null,null,null,null,null,null,null,null,2040.163086,544.831177,160.03125,850.0,100.0,true,"""BombsiteA""","""frozen""",76561198068422762,"""FaZe Clan""","""t""",0.0,false,false,"""Natus Vincere""","""ct""",23.735229,0,88,0,true,"""head""",false,false,0,0,"""FaZe Clan""","""t""",true,9960,0,2581.278076,1305.825684,160.03125,700,12,true,"""Library""","""Aleksib""",76561198013243326,"""Natus Vincere""","""ct""",0,"""glock""","""17293822569102901252""","""36694885152""","""""",0,1
false,278.091522,1736.573242,120.855316,850.0,64.0,true,"""Banana""","""jL""",76561198176878303,"""Natus Vincere""","""ct""",0.0,810.15509,-554.923035,96.286346,950.0,100.0,true,"""BackAlley""","""w0nderful""",76561199063068840,"""Natus Vincere""","""ct""",0.0,false,false,"""Natus Vincere""","""ct""",5.576701,0,137,0,true,"""head""",false,false,0,0,"""FaZe Clan""","""t""",false,10734,0,1002.924255,-647.492981,97.546738,800,73,true,"""Apartments""","""karrigan""",76561197989430253,"""FaZe Clan""","""t""",0,"""elite""","""17293822569151660034""","""15846704275""","""""",0,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
false,null,null,null,null,null,null,null,null,null,null,null,null,2000.276978,526.535034,160.03125,4350.0,100.0,true,"

In [ ]:
ticks = dem.ticks
ticks.tail(5)

In [16]:
def calculate_multikill_rounds(dem) -> pl.DataFrame:

    # Ensure 'dem.kills' is not empty before proceeding
    if dem.kills is None or dem.kills.shape[0] == 0:
        return pl.DataFrame({"steamid": pl.Series(dtype=pl.UInt64),
                             "2k": pl.Series(dtype=pl.UInt32),
                             "3k": pl.Series(dtype=pl.UInt32),
                             "4k": pl.Series(dtype=pl.UInt32),
                             "5k": pl.Series(dtype=pl.UInt32)})

    kills_per_player_per_round = dem.kills.group_by(["round_num", "attacker_steamid"]).agg(
        pl.len().alias("kills_in_round")
    )

    multikill_counts = kills_per_player_per_round.group_by("attacker_steamid").agg(
        pl.when(pl.col("kills_in_round") == 2).then(1).otherwise(0).sum().alias("2k"),
        pl.when(pl.col("kills_in_round") == 3).then(1).otherwise(0).sum().alias("3k"),
        pl.when(pl.col("kills_in_round") == 4).then(1).otherwise(0).sum().alias("4k"),
        pl.when(pl.col("kills_in_round") == 5).then(1).otherwise(0).sum().alias("5k")
    )

    multikill_counts = multikill_counts.rename({"attacker_steamid": "steamid"})

    return multikill_counts


In [17]:
multikill_df = calculate_multikill_rounds(dem)
multikill_df

steamid,2k,3k,4k,5k
u64,i32,i32,i32,i32
76561198176878303,2,0,1,0
76561198201620490,2,0,0,0
76561198050250233,1,1,0,0
76561197991272318,0,1,0,0
76561198068422762,1,0,0,0
76561198013243326,3,0,0,0
76561197989430253,0,1,0,0
76561199063068840,2,1,0,0
76561197997351207,4,1,0,0


In [13]:
clutches_data = []

all_players = dem.ticks.select(["steamid", "name"]).unique()
player_name_map = {row["steamid"]: row["name"] for row in all_players.iter_rows(named=True)}

for round_info in dem.rounds.iter_rows(named=True):
    round_num = round_info['round_num']
    round_start_tick = round_info['start']
    round_end_tick = round_info['end']
    winning_team = round_info['winner'] # Ex: 'CT' or 'T'

    round_ticks_df = dem.ticks.filter(
        (pl.col("tick") >= round_start_tick) & (pl.col("tick") <= round_end_tick)
    )
    round_kills_df = dem.kills.filter(
        (pl.col("round_num") == round_num) &
        (pl.col("tick") >= round_start_tick) & (pl.col("tick") <= round_end_tick)
    ).sort("tick")

    alive_counts_per_tick = round_ticks_df.group_by("tick").agg(
        (pl.when(pl.col("side") == "ct").then(pl.col("is_alive")).otherwise(0)).sum().alias("ct_alive"),
        (pl.when(pl.col("side") == "t").then(pl.col("is_alive")).otherwise(0)).sum().alias("t_alive")
    )
    
    players_alive_at_tick = round_ticks_df.group_by("tick").agg(
        pl.struct(["steamid", "is_alive", "side"]).alias("players_state")
    )
    
    merged_round_data = alive_counts_per_tick.join(
        players_alive_at_tick, on="tick", how="left"
    ).sort("tick")


    clutch_active = False
    clutcher_steamid = None
    clutcher_team_side = None
    clutch_start_tick = None
    opponents_at_start = 0

    for i, tick_data in enumerate(merged_round_data.iter_rows(named=True)):
        current_tick = tick_data['tick']
        ct_alive = tick_data['ct_alive']
        t_alive = tick_data['t_alive']
        players_state = tick_data['players_state']
        
        is_clutch_condition_met = (ct_alive == 1 and t_alive >= 2) or \
                                    (t_alive == 1 and ct_alive >= 2)

        clutcher_is_alive_this_tick = False
        if clutch_active and clutcher_steamid:
            for p_state in players_state:
                if p_state['steamid'] == clutcher_steamid and p_state['is_alive']:
                    clutcher_is_alive_this_tick = True
                    break

        if is_clutch_condition_met:
            if not clutch_active: # Start of a NEW clutch
                clutch_active = True
                clutch_start_tick = current_tick
                
                if ct_alive == 1:
                    clutcher_team_side = 'ct'
                    opponents_at_start = t_alive # Capture opponents at the start
                else: # t_alive == 1
                    clutcher_team_side = 't'
                    opponents_at_start = ct_alive # Capture opponents at the start
                
                for p_state in players_state:
                    if p_state['is_alive'] and p_state['side'] == clutcher_team_side:
                        clutcher_steamid = p_state['steamid']
                        break
        
        else: # Clutch condition (1vX, X>=2) is NO longer met in this tick
            if clutch_active: # But there was an active clutch in the previous tick
                # The clutch ends if the clutcher dies
                if not clutcher_is_alive_this_tick:
                    clutch_end_tick = current_tick # Clutch ends with the clutcher's death
                    
                    # --- CLUTCH OUTCOME: "lost" ---
                    clutch_outcome = "lost" # Clutcher died

                    final_clutch_kills = 0
                    if clutcher_steamid:
                        final_clutch_kills = round_kills_df.filter(
                            (pl.col("tick") >= clutch_start_tick) &
                            (pl.col("tick") <= clutch_end_tick) &
                            (pl.col("attacker_steamid") == clutcher_steamid)
                        ).shape[0]

                    # Determine if the clutcher's team won the round
                    round_team_won = (clutcher_team_side == winning_team.lower()) 

                    # Clutcher did not survive the round if they died to end the clutch
                    clutcher_survived_round = False

                    clutches_data.append({
                        "round_num": round_num,
                        "clutcher_steamid": clutcher_steamid,
                        "clutcher_name": player_name_map.get(clutcher_steamid, "Unknown"),
                        "clutcher_team_side": clutcher_team_side,
                        "opponents_at_start": opponents_at_start,
                        "clutch_start_tick": clutch_start_tick,
                        "clutch_end_tick": clutch_end_tick,
                        "clutch_kills": final_clutch_kills,
                        "clutch_outcome": clutch_outcome, # Renomeado
                        "round_won": round_team_won,
                        "clutcher_survived_round": clutcher_survived_round
                    })
                    
                    clutch_active = False
                    clutcher_steamid = None
                    clutcher_team_side = None
                    clutch_start_tick = None
                    opponents_at_start = 0

    # --- Round end processing logic ---
    if clutch_active:
        clutch_end_tick = round_end_tick 
        
        final_clutch_kills = 0
        if clutcher_steamid:
            final_clutch_kills = round_kills_df.filter(
                (pl.col("tick") >= clutch_start_tick) &
                (pl.col("tick") <= clutch_end_tick) &
                (pl.col("attacker_steamid") == clutcher_steamid)
            ).shape[0]

        # Determine if the clutcher's team won the round
        round_team_won = (clutcher_team_side == winning_team.lower())

        # NOVO: Determinar o clutch_outcome para o final da rodada
        if round_team_won:
            # --- CLUTCH OUTCOME: "won" ---
            clutch_outcome = "won" # Clutcher's team won the round
        else:
            # --- CLUTCH OUTCOME: "save" ---
            clutch_outcome = "save" # Clutcher survived, but team lost the round (e.g., time expired, bomb detonated for opponents)
        
        # Determine if the clutcher survived the round (always True if we reach this block)
        # We already know clutcher is alive at this point because `clutch_active` is True
        # and they weren't caught by the `if not clutcher_is_alive_this_tick` block.
        clutcher_survived_round = True # If clutch_active is true at end of round, clutcher survived.

        clutches_data.append({
            "round_num": round_num,
            "clutcher_steamid": clutcher_steamid,
            "clutcher_name": player_name_map.get(clutcher_steamid, "Unknown"),
            "clutcher_team_side": clutcher_team_side,
            "opponents_at_start": opponents_at_start,
            "clutch_start_tick": clutch_start_tick,
            "clutch_end_tick": clutch_end_tick,
            "clutch_kills": final_clutch_kills,
            "clutch_outcome": clutch_outcome, # Renomeado
            "round_won": round_team_won,
            "clutcher_survived_round": clutcher_survived_round
        })
        
        clutch_active = False
        clutcher_steamid = None
        clutcher_team_side = None
        clutch_start_tick = None
        opponents_at_start = 0

clutches_df = pl.DataFrame(clutches_data)

In [14]:
pl.Config.set_tbl_rows(20)
clutches_df

round_num,clutcher_steamid,clutcher_name,clutcher_team_side,opponents_at_start,clutch_start_tick,clutch_end_tick,clutch_kills,clutch_outcome,round_won,clutcher_survived_round
i64,i64,str,str,i64,i64,i64,i64,str,bool,bool
1,76561199063068840,"""w0nderful""","""ct""",2,12017,12448,0,"""save""",false,true
3,76561198068422762,"""frozen""","""t""",5,28567,29749,0,"""lost""",false,false
4,76561197991272318,"""ropz""","""t""",4,36701,38837,0,"""save""",false,true
5,76561197989430253,"""karrigan""","""t""",5,47176,47343,1,"""lost""",false,false
6,76561198013243326,"""Aleksib""","""ct""",4,56609,58805,1,"""save""",false,true
7,76561197997351207,"""rain""","""t""",2,69151,69524,0,"""lost""",false,false
8,76561199063068840,"""w0nderful""","""ct""",2,77268,78962,0,"""lost""",false,false
9,76561198201620490,"""broky""","""t""",5,84443,88050,2,"""save""",false,true
10,76561197989430253,"""karrigan""","""t""",5,95545,95721,1,"""lost""",false,false


In [7]:
def calculate_clutches(dem) -> pl.DataFrame:
    clutches_data = []

    all_players = dem.ticks.select(["steamid", "name"]).unique()
    player_name_map = {row["steamid"]: row["name"] for row in all_players.iter_rows(named=True)}

    for round_info in dem.rounds.iter_rows(named=True):
        round_num = round_info['round_num']
        round_start_tick = round_info['start']
        round_end_tick = round_info['end']
        winning_team = round_info['winner'] # Ex: 'CT' ou 'T'

        # Filtrar ticks e kills para a rodada atual
        round_ticks_df = dem.ticks.filter(
            (pl.col("tick") >= round_start_tick) & (pl.col("tick") <= round_end_tick)
        )
        round_kills_df = dem.kills.filter(
            (pl.col("round_num") == round_num) &
            (pl.col("tick") >= round_start_tick) & (pl.col("tick") <= round_end_tick)
        ).sort("tick")

        # Preparar os dados de jogadores vivos por tick para a rodada
        alive_counts_per_tick = round_ticks_df.group_by("tick").agg(
            (pl.when(pl.col("side") == "ct").then(pl.col("is_alive")).otherwise(0)).sum().alias("ct_alive"),
            (pl.when(pl.col("side") == "t").then(pl.col("is_alive")).otherwise(0)).sum().alias("t_alive")
        )
        
        players_alive_at_tick = round_ticks_df.group_by("tick").agg(
            pl.struct(["steamid", "is_alive", "side"]).alias("players_state")
        )
        
        # Juntar os dados e garantir a ordenação por tick
        merged_round_data = alive_counts_per_tick.join(
            players_alive_at_tick, on="tick", how="left"
        ).sort("tick")


        # Variáveis de controle do estado do clutch
        clutch_active = False
        clutcher_steamid = None
        clutcher_team_side = None
        clutch_start_tick = None
        opponents_at_start = 0

        for i, tick_data in enumerate(merged_round_data.iter_rows(named=True)):
            current_tick = tick_data['tick']
            ct_alive = tick_data['ct_alive']
            t_alive = tick_data['t_alive']
            players_state = tick_data['players_state']
            
            is_clutch_condition_met = (ct_alive == 1 and t_alive >= 2) or \
                                      (t_alive == 1 and ct_alive >= 2)

            clutcher_is_alive_this_tick = False
            if clutch_active and clutcher_steamid:
                for p_state in players_state:
                    if p_state['steamid'] == clutcher_steamid and p_state['is_alive']:
                        clutcher_is_alive_this_tick = True
                        break

            if is_clutch_condition_met:
                if not clutch_active: # Início de um NOVO clutch
                    clutch_active = True
                    clutch_start_tick = current_tick
                    
                    if ct_alive == 1:
                        clutcher_team_side = 'ct'
                        opponents_at_start = t_alive # Captura oponentes no início
                    else: # t_alive == 1
                        clutcher_team_side = 't'
                        opponents_at_start = ct_alive # Captura oponentes no início
                    
                    for p_state in players_state:
                        if p_state['is_alive'] and p_state['side'] == clutcher_team_side:
                            clutcher_steamid = p_state['steamid']
                            break
            
            else: # A condição de clutch (1vX, X>=2) NÃO é mais atendida neste tick
                if clutch_active: # Mas havia um clutch ativo no tick anterior
                    # O clutch termina se o clutcher morreu
                    if not clutcher_is_alive_this_tick:
                        clutch_end_tick = current_tick # Clutch termina com a morte do clutcher
                        clutch_won = False # Clutch perdido por morte
                        
                        final_clutch_kills = 0
                        if clutcher_steamid:
                            final_clutch_kills = round_kills_df.filter(
                                (pl.col("tick") >= clutch_start_tick) &
                                (pl.col("tick") <= clutch_end_tick) &
                                (pl.col("attacker_steamid") == clutcher_steamid)
                            ).shape[0]

                        # Determina se o time do clutcher venceu a rodada
                        round_team_won = (clutcher_team_side == winning_team.lower()) 

                        # NOVO: Clutcher não sobreviveu ao round se ele morreu para encerrar o clutch
                        clutcher_survived_round = False

                        clutches_data.append({
                            "round_num": round_num,
                            "clutcher_steamid": clutcher_steamid,
                            "clutcher_name": player_name_map.get(clutcher_steamid, "Unknown"),
                            "clutcher_team_side": clutcher_team_side,
                            "opponents_at_start": opponents_at_start,
                            "clutch_start_tick": clutch_start_tick,
                            "clutch_end_tick": clutch_end_tick,
                            "clutch_kills": final_clutch_kills,
                            "clutch_won": clutch_won,
                            "round_won": round_team_won,
                            "clutcher_survived_round": clutcher_survived_round # Nova coluna
                        })
                        
                        clutch_active = False
                        clutcher_steamid = None
                        clutcher_team_side = None
                        clutch_start_tick = None
                        opponents_at_start = 0

        # --- Lógica de encerramento no final da rodada ---
        if clutch_active:
            clutch_end_tick = round_end_tick 
            
            final_clutch_kills = 0
            if clutcher_steamid:
                final_clutch_kills = round_kills_df.filter(
                    (pl.col("tick") >= clutch_start_tick) &
                    (pl.col("tick") <= clutch_end_tick) &
                    (pl.col("attacker_steamid") == clutcher_steamid)
                ).shape[0]

            clutch_won = False
            
            # Determina se o time do clutcher venceu a rodada
            round_team_won = (clutcher_team_side == winning_team.lower())

            # Se o clutcher chegou até o final da rodada com o clutch ativo,
            # ele venceu o clutch se o time dele venceu a rodada.
            if round_team_won:
                clutch_won = True
            
            # NOVO: Determinar se o clutcher sobreviveu ao round
            clutcher_survived_round = False
            if clutcher_steamid:
                # Busca o último estado de vida do clutcher até o round_end_tick
                last_known_state = round_ticks_df.filter(
                    (pl.col("tick") <= round_end_tick) & 
                    (pl.col("steamid") == clutcher_steamid)
                ).sort("tick", descending=True).select("is_alive").head(1) 
                
                if last_known_state.height == 1:
                    clutcher_survived_round = last_known_state.item()
            
            clutches_data.append({
                "round_num": round_num,
                "clutcher_steamid": clutcher_steamid,
                "clutcher_name": player_name_map.get(clutcher_steamid, "Unknown"),
                "clutcher_team_side": clutcher_team_side,
                "opponents_at_start": opponents_at_start,
                "clutch_start_tick": clutch_start_tick,
                "clutch_end_tick": clutch_end_tick,
                "clutch_kills": final_clutch_kills,
                "clutch_won": clutch_won,
                "round_won": round_team_won,
                "clutcher_survived_round": clutcher_survived_round # Nova coluna
            })
            
            clutch_active = False
            clutcher_steamid = None
            clutcher_team_side = None
            clutch_start_tick = None
            opponents_at_start = 0

    clutches_df = pl.DataFrame(clutches_data)
    return clutches_df